In [1]:
import os
%pwd

'c:\\Users\\Prarthana\\Desktop\\projects\\noise_reduction_autoencoder\\research'

In [2]:
os.chdir('../')
%pwd

'c:\\Users\\Prarthana\\Desktop\\projects\\noise_reduction_autoencoder'

In [ ]:
#update entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params: dict

In [4]:
#update configuration manager
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.utils.common import read_yaml, create_directories
#from src.entity.config_entity import DataIngestionConfig, PrepareBaseModelConfig
from pathlib import Path

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([Path(config.root_dir)])
        return PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params=self.params
        )

In [8]:
#update components
import tensorflow as tf
from pathlib import Path
from src import logger
from src.entity.config_entity import PrepareBaseModelConfig

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        """Initialize with configuration."""
        self.config = config

    def get_base_model(self):
        """Create the base autoencoder model."""
        try:
            self.model = self._build_autoencoder()
            self.save_model(path=self.config.base_model_path, model=self.model)
            logger.info("Base autoencoder model created and saved")
        except Exception as e:
            logger.error(f"Failed to create base model: {e}")
            raise e

    def _build_autoencoder(self):
        """Build the convolutional autoencoder architecture."""
        input_img = tf.keras.Input(shape=(self.config.params["IMAGE_SIZE"][0],
                                       self.config.params["IMAGE_SIZE"][1], 1))
        # Encoder
        x = tf.keras.layers.Conv2D(self.config.params["MODEL"]["FILTERS"][0],
                                 self.config.params["MODEL"]["KERNEL_SIZE"],
                                 activation='relu', padding='same')(input_img)
        x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
        x = tf.keras.layers.Conv2D(self.config.params["MODEL"]["FILTERS"][1],
                                 self.config.params["MODEL"]["KERNEL_SIZE"],
                                 activation='relu', padding='same')(x)
        encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
        # Decoder
        x = tf.keras.layers.Conv2D(self.config.params["MODEL"]["FILTERS"][1],
                                 self.config.params["MODEL"]["KERNEL_SIZE"],
                                 activation='relu', padding='same')(encoded)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
        x = tf.keras.layers.Conv2D(self.config.params["MODEL"]["FILTERS"][0],
                                 self.config.params["MODEL"]["KERNEL_SIZE"],
                                 activation='relu', padding='same')(x)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
        decoded = tf.keras.layers.Conv2D(1, self.config.params["MODEL"]["KERNEL_SIZE"],
                                      activation='sigmoid', padding='same')(x)
        autoencoder = tf.keras.Model(input_img, decoded)
        return autoencoder

    @staticmethod
    def _prepare_full_model(model, learning_rate):
        """Compile the autoencoder model."""
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=["mae"]  # Mean Absolute Error for monitoring
        )
        model.summary(print_fn=lambda x: logger.info(x.encode('ascii', 'ignore').decode('ascii')))

        return model

    def update_base_model(self):
        """Prepare and save the full autoencoder model."""
        try:
            self.full_model = self._prepare_full_model(
                model=self.model,
                learning_rate=self.config.params["MODEL"]["LEARNING_RATE"]
            )
            self.save_model(path=self.config.base_model_path, model=self.full_model)
            logger.info("Full autoencoder model updated and saved")
        except Exception as e:
            logger.error(f"Failed to update base model: {e}")
            raise e

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """Save the model to the specified path."""
        try:
            model.save(path)
            logger.info(f"Model saved to {path}")
        except Exception as e:
            logger.error(f"Failed to save model: {e}")
            raise e

In [9]:
#update pipeline
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    logger.error(f"Error in prepare base model: {e}", exc_info=True)
    raise e

[2025-04-22 00:17:52,124: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-04-22 00:17:52,129: INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-22 00:17:52,131: INFO: common: Created directory at: artifacts]
[2025-04-22 00:17:52,133: INFO: common: Created directory at: artifacts\models]


[2025-04-22 00:17:52,204: INFO: 2279715737: Model saved to artifacts\models\base_autoencoder.keras]
[2025-04-22 00:17:52,205: INFO: 2279715737: Base autoencoder model created and saved]


[2025-04-22 00:17:52,218: INFO: 2279715737: Model: "functional_1"

 Layer (type)                     Output Shape                  Param # 

 input_layer_1 (InputLayer)       (None, 128, 128, 1)                 0 

 conv2d_5 (Conv2D)                (None, 128, 128, 32)              320 

 max_pooling2d_2 (MaxPooling2D)   (None, 64, 64, 32)                  0 

 conv2d_6 (Conv2D)                (None, 64, 64, 64)             18,496 

 max_pooling2d_3 (MaxPooling2D)   (None, 32, 32, 64)                  0 

 conv2d_7 (Conv2D)                (None, 32, 32, 64)             36,928 

 up_sampling2d_2 (UpSampling2D)   (None, 64, 64, 64)                  0 

 conv2d_8 (Conv2D)                (None, 64, 64, 32)             18,464 

 up_sampling2d_3 (UpSampling2D)   (None, 128, 128, 32)                0 

 conv2d_9 (Conv2D)                (None, 128, 128, 1)               289 

 Total params: 74,497 (291.00 KB)
 Trainable params: 74,497 (291.00 KB)
 Non-trainable params: 0 (0.00 B)
]
[2025-04-22